**Импорт библиотек**

In [69]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import random
import torch.nn.functional as F

**Загрузка датасетов**

In [72]:
interactions = pd.read_parquet("interactions.parquet")
tracks = pd.read_parquet("tracks.parquet")
catalog_names = pd.read_parquet("catalog_names.parquet")

**Обрежу самыый большой датасет для скорости**

In [75]:
interactions = interactions.iloc[0:100_000]

In [77]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 45
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     100000 non-null  int32         
 1   track_id    100000 non-null  int32         
 2   track_seq   100000 non-null  int16         
 3   started_at  100000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 2.5 MB


**Сортирую track_seq и групирую по user_id**

In [80]:
interactions = interactions.sort_values(by=['user_id', 'track_seq'])
user_sequences = interactions.groupby('user_id')['track_id'].apply(list).reset_index()

In [82]:
interactions.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [84]:
user_sequences.head()

,user_id,track_id
0,0,"[99262, 589498, 590262, 590303, 590692, 590803..."
1,1,"[24417, 108208, 108209, 592642, 628687, 733449..."
2,2,"[264937, 672689, 4321285, 5335351, 5658525, 58..."
3,3,"[6006252, 21642261, 21642265, 24692821, 259952..."
4,4,"[966, 4094, 9760, 9769, 18392, 19042, 21184, 2..."


Сортируем все уникальные треки в **all_track** и создаем **MASK_TOKEN** для маскировки токенов в последовательности. **PAD** токены нужны просто как плейсхолдеры для того что бы каждая последовательность была одинаковой длинны. 

Дальше мы создаем словарь где ключем будет **track_id** а **value** порядковым номером в словаре. Так же создаем инверсивный словарь для обратного получения **track_id**.

In [87]:
all_tracks = interactions['track_id'].unique().tolist()
MASK_TOKEN = "<MASK>"
PAD_TOKEN = "<PAD>"

vocab = {PAD_TOKEN: 0, MASK_TOKEN: 1}
for track in all_tracks:
    if track not in vocab:
        vocab[track] = len(vocab)

vocab_size = len(vocab)
inv_vocab = {v: k for k, v in vocab.items()}

sequences: список списков осстоящих из track_id пользователей 
vocab: track_id -> index
mask_prob: вероятность замаскировать токен

_ _ len _ _ :  возвращает длину **sequences**. #это нужно для  PyTorch датасетов

_ _ getitem _ _ : сама функция используется внутри PyTorch (потому что это map-styled dataset). 
1) Преобразуем все **id**  треков в sequence в индексы, а если трека нет в vocab то в **PAD** токен.
2) Потом если длина **sequence** больше максимальной задданной длины, мы укарачиваем sequence. В обратном случае мы дополняем ее **PAD** токенами.
3) Дальше мы создаем **target** последовательность и начальную последовательность, в которой мы маскируем с шансом **mask_prob** токены в **MASK_TOKEN**.
4) Делаем из каждой последовательности тензор для **PyTorch**.

In [114]:
class BERT4RecDataset(Dataset):
    def __init__(self, sequences, vocab, max_seq_len=50, mask_prob = 0.7):
        self.sequences = sequences
        self.vocab = vocab
        self.max_seq_len = max_seq_len
        self.mask_prob = mask_prob

    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        seq = self.sequences[idx]
        seq_idx = [self.vocab.get(token, self.vocab[PAD_TOKEN]) for token in seq]
        
        if len(seq_idx) >= self.max_seq_len:
            seq_idx = seq_idx[-self.max_seq_len:]
        else:
            seq_idx = [self.vocab[PAD_TOKEN]] * (self.max_seq_len - len(seq_idx)) + seq_idx
        
        input_seq = list(seq_idx)
        target_seq = list(seq_idx)
        
        for i in range(len(input_seq)):
            if input_seq[i] != self.vocab[PAD_TOKEN] and random.random() < self.mask_prob:
                input_seq[i] = self.vocab[MASK_TOKEN]
        
        input_seq = torch.tensor(input_seq, dtype=torch.long)
        target_seq = torch.tensor(target_seq, dtype=torch.long)
        
        return input_seq, target_seq

from sklearn.model_selection import train_test_split
sequences = user_sequences['track_id'].tolist()
train_sequences, val_sequences = train_test_split(sequences, test_size = 0.2, random_state = 42)
train_dataset = BERT4RecDataset(train_sequences, vocab, max_seq_len=50, mask_prob=0.7)
val_dataset = BERT4RecDataset(val_sequences, vocab, max_seq_len=50, mask_prob=0.7)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) 
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

Здесь мы описываем модель, сначала идет конструктор со строкой super... которая вызывает конструктор nn.Module (это нужно для коректной работы объявления следущих переменых)

1) дальше идет функция forward которая принимает на вход тензор x (batch_size, seq_len).
2) positions это просто тензор с нумерацией мест по типу ((0, 1),(0, 1)) и.т.д. Потом мы прогоняем positions через nn.Embedding присваивая по началу рандомные значения для векторов(это изменится на моменте оптимизатора и бекпропагейшн).
3) Далее мы складываем позиционные эмбединги с эмбедингами токенов и в последствии нормируем, что бы значения были в равных пределах (от 0 до 1). И далее делаем дропаут для добавления случайного шума (это помогает модели не переобучаться).

In [117]:
class BERT4Rec(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=8, num_layers=4, max_seq_len=50, dropout=0.1): 
        
        super(BERT4Rec, self).__init__()
        self.embed_dim = embed_dim
        self.token_embeddings = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.position_embeddings = nn.Embedding(max_seq_len, embed_dim)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
        
        self.output_layer = nn.Linear(embed_dim, vocab_size)
        
    def forward(self, x):
        batch_size, seq_len = x.size()
        token_emb = self.token_embeddings(x)
        
        positions = torch.arange(seq_len, device=x.device).unsqueeze(0).expand(batch_size, seq_len)
        pos_emb = self.position_embeddings(positions)
        
        x = token_emb + pos_emb
        x = self.layer_norm(x)
        x = self.dropout(x)
        
       
        x = x.transpose(0, 1) #[seq_len, batch_size, embed_dim]
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)  #[B, seq_len, embed_dim]
        
        logits = self.output_layer(x)  # [B, seq_len, vocab_size]
        return logits

# Создаем модель
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BERT4Rec(vocab_size=vocab_size, embed_dim=128, num_heads=4, num_layers=2, max_seq_len=50, dropout=0.1).to(device)

In [120]:
def compute_recall_ndcg(model, dataloader, k=10, device="cpu"):
    model.eval()
    total_hits = 0  # для recall
    total_ndcg = 0.0  # для NDCG
    total_count = 0  # общее число позиций (без PAD)
    
    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            logits = model(inputs)  # [B, seq_len, vocab_size]
            topk = logits.topk(k, dim=-1).indices  # [B, seq_len, k]
            
            batch_size, seq_len = targets.size()
            for i in range(batch_size):
                for j in range(seq_len):
                    true_token = targets[i, j].item()
                    if true_token == 0:
                        continue
                    total_count += 1
                    topk_tokens = topk[i, j].tolist()
                    if true_token in topk_tokens:
                        total_hits += 1
                        rank = topk_tokens.index(true_token) + 1
                        total_ndcg += 1 / np.log2(rank + 1)
    
    recall = total_hits / total_count if total_count > 0 else 0
    ndcg = total_ndcg / total_count if total_count > 0 else 0
    return recall, ndcg

In [122]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=vocab[PAD_TOKEN])

num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = outputs.view(-1, vocab_size)
        targets = targets.view(-1)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

    val_recall, val_ndcg = compute_recall_ndcg(model, val_loader, k=10, device=device)
    print(f"Validation Recall@10: {val_recall:.4f}, NDCG@10: {val_ndcg:.4f}")

Epoch 1/30, Loss: 11.0097
Validation Recall@10: 0.0022, NDCG@10: 0.0011
Epoch 2/30, Loss: 10.3269
Validation Recall@10: 0.0102, NDCG@10: 0.0059
Epoch 3/30, Loss: 9.7263
Validation Recall@10: 0.0285, NDCG@10: 0.0194
Epoch 4/30, Loss: 9.2861
Validation Recall@10: 0.0369, NDCG@10: 0.0280
Epoch 5/30, Loss: 8.9433
Validation Recall@10: 0.0535, NDCG@10: 0.0400
Epoch 6/30, Loss: 8.6516
Validation Recall@10: 0.0642, NDCG@10: 0.0501
Epoch 7/30, Loss: 8.3700
Validation Recall@10: 0.0770, NDCG@10: 0.0602
Epoch 8/30, Loss: 8.0994
Validation Recall@10: 0.0879, NDCG@10: 0.0683
Epoch 9/30, Loss: 7.8117
Validation Recall@10: 0.1026, NDCG@10: 0.0795
Epoch 10/30, Loss: 7.5210
Validation Recall@10: 0.1147, NDCG@10: 0.0903
Epoch 11/30, Loss: 7.2242
Validation Recall@10: 0.1243, NDCG@10: 0.1002
Epoch 12/30, Loss: 6.9213
Validation Recall@10: 0.1385, NDCG@10: 0.1130
Epoch 13/30, Loss: 6.6211
Validation Recall@10: 0.1528, NDCG@10: 0.1244
Epoch 14/30, Loss: 6.3218
Validation Recall@10: 0.1553, NDCG@10: 0.1267

In [105]:
def get_track_name(track_id, catalog_names_df):
    row = catalog_names_df[
        (catalog_names_df['id'] == track_id) &
        (catalog_names_df['type'] == 'track')
    ]
    if not row.empty:
        return row['name'].values[0]
    else:
        return "Unknown Track"

In [108]:
def convert_track_ids_to_names(track_ids, catalog_names_df):
    names = []
    for track_id in track_ids:
        if track_id in [MASK_TOKEN, PAD_TOKEN]:
            names.append(track_id)
            continue
        row = catalog_names_df[
            (catalog_names_df['id'] == track_id) & 
            (catalog_names_df['type'] == 'track')
        ]
        if not row.empty:
            names.append(row['name'].values[0])
        else:
            names.append("Unknown Track")
    return names

In [110]:
def predict_masked(model, sequence, vocab, max_seq_len=50):
    model.eval()
    seq_idx = [vocab.get(token, vocab[PAD_TOKEN]) for token in sequence]
    if len(seq_idx) >= max_seq_len:
        seq_idx = seq_idx[-max_seq_len:]
    else:
        seq_idx = [vocab[PAD_TOKEN]] * (max_seq_len - len(seq_idx)) + seq_idx
    input_seq = torch.tensor(seq_idx, dtype=torch.long).unsqueeze(0).to(device)
    
    with torch.no_grad():
        logits = model(input_seq)

    pred_tokens = logits.argmax(dim=-1).squeeze(0).cpu().numpy()
    return pred_tokens


sample_seq = sequences[12]

sample_seq_masked = sample_seq[:-1] + [MASK_TOKEN]
predicted = predict_masked(model, sample_seq_masked, vocab)
predicted_token_id = predicted[-1]
predicted_track_id = inv_vocab[predicted_token_id]
track_name = get_track_name(predicted_track_id, catalog_names)
track_names = convert_track_ids_to_names(sample_seq, catalog_names)
print("Initial sequence (names):")
for name in track_names:
    print("-", name)
print("Predicted Track Name:", track_name)

Исходная последовательность (названия):
- Boulevard of Broken Dreams
- Rise Up
- Long Nights
- Night: Part I. Snow
- Faded
- Are You With Me
- Никому не отдам
- Stressed Out
- Here She Comes Again
- The Hills
- I Took A Pill In Ibiza
- Ты не такой
- Грею счастье
- Faded
- Bludfire
- Hymn For The Weekend
- California
- Boom Boom Boom
- Don't Let Me Down
- Cheap Thrills
- This Girl
- Save Me
- Feel
- You Are The Only One
- HandClap
- Мегахит
- The Ocean
- Непохожие
- Выпускной (Медлячок)
- Кружит
- Sing Me to Sleep
- Sweet Harmony (feat. Pearl Andersson)
- Perfect Strangers
- Tuesday
- I Never Felt so Right
- Wide Awake
- Lost on You
- Пусть весь мир подождёт
- Lordly
- Вдвоём
- My Way
- Rockabye
Predicted Track Name: FIND YOUR WAY BACK
